# Open a microscopy data in any format 

This notebook enables the user to load microscopy data in different formats listed [here](https://bio-formats.readthedocs.io/en/v6.12.0/supported-formats.html) in a similar fashion. It also enables the reading of the metadata. The installation of the package aicsimageio is required. Documentation can be found [here](https://allencellmodeling.github.io/aicsimageio/index.html).
The following steps are required for the installation of the required package: <br>
> \$ conda activate \<env\> <br>
    \$ pip install aicsimageio <br>
    \$ pip install aicsimageio[base-imageio,nd2,bfio,all] <br>
    \$ pip install aicsimageio readlif>=0.6.4 <br>
    \$ pip install aicsimageio aicspylibczi>=3.0.5 fsspec>=2022.7.1 <br>
    \$ conda install -c conda-forge bioformats_jar <br> 

Load the image with its metadata in this next cell by replacing the image path directory in that cell. Make sure to have the path correct and use / instead of \

In [1]:
# EXAMPLE
from scyjava import config
from aicsimageio import AICSImage
mydata = AICSImage("Y:/April05/FluoCells1_BPAE_F36924/FluoCells1_F36924_001.nd2") # THE IMAGE PATH DIRECTORY HERE

The following packages need to be imported to display the image and extract the metadata from the data.

In [2]:
# IMPORT PACKAGES
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Next, we read the image and the metadata. Metadata includes the number of channels in the images, the name of each channel and emission wavelength associated with each channel, the voxel size and physical size of the image. Many other acquisition parameters are stored in the metadata and we can access them from here.

In [3]:
# READ IMAGE AND THE METADATA
metadata = mydata.metadata 
myimg = mydata.data
imgsize = myimg.shape

metadata = mydata.metadata
channumber=len(metadata['metadata'].channels) # or simply imgsize[1]
channelname=[metadata['metadata'].channels[c].channel.name for c in np.arange(channumber)] # 
wavelengthlist=[metadata['metadata'].channels[c].channel.emissionLambdaNm for c in np.arange(channumber)] # emission wavelength in each channel
voxelsize = metadata['metadata'].channels[0].volume.axesCalibration # voxel size in um
voxelcount = metadata['metadata'].channels[0].volume.voxelCount # 3D size
NA = metadata['metadata'].channels[0].microscope.objectiveNumericalAperture
ri_immersion = metadata['metadata'].channels[0].microscope.immersionRefractiveIndex # refractive index of the immersion medium 
microscope_name = metadata['metadata'].channels[0].microscope.objectiveName
Acquisition = metadata['text_info']['capturing']  # contains info about the exposure time of each channel 
# print(Acquisition)
print('Data recorded from:', microscope_name)
print('Voxel size:', voxelsize,'in micrometer')
print('Size XYZ of the image:', voxelcount)
print('Number of channels:', channumber)
print('Time frames:', imgsize[0])

df = pd.DataFrame(columns=channelname,dtype=object)
data = {channelname[c]:wavelengthlist[c] for c in np.arange(channumber)}
df = pd.DataFrame(data=data,index=['Emission wavelength [in nm]'])
df

Data recorded from: Plan Apo VC 100x Oil DIC N2
Voxel size: [0.065, 0.065, 0.1] in micrometer
Size XYZ of the image: [2048, 2044, 101]
Number of channels: 3
Time frames: 1


,Red Camera,Green Camera,Blue Camera
Emission wavelength [in nm],620.0,525.0,460.0


The image is usually in 5D where the first three dimensions are spatials, the channel and timeframe are in the fourth and fifth dimension respectively. In the next cell, we generate a list of colormaps for displaying each channel. Users are advised to choose their desired colormaps based on their data and number of channels in the data if they can but otherwise we assume the first three existing channels are associated with RGB (red, green, blue) colors.

In [4]:
# COLORMAP FOR DISPLAYING DIFFERENT CHANNELS 
if channumber<=3:
    cmpsuggestion=['hot','viridis','gnuplot2'] # for red, green and blue channel
else:
    cmplist=plt.colormaps() # list of the colormaps available on python
    cmpsuggestion=['hot','viridis','gnuplot2']
    for c in np.arange(channumber-3):
        cmpsuggestion.append(cmplist[c]) 

The image is displayed using the code in the next cell. The display is interactive where the user can select the channel to display and slide over the axial (z) position and time frame in the data. The combination of all the channels is displayed below the display of the selected channel.

In [10]:
# DISPLAY THE IMAGES

def series(time, zpos, channel):
    channels = {channelname[c]:c for c in np.arange(channumber)}
    listmap={channelname[c]:cmpsuggestion[c] for c in np.arange(channumber)}
    
    fig, ax = plt.subplots()
    cpos=channels[channel]
    datadisp = ax.imshow(myimg[time,channels[channel],zpos,:,:],cmap=listmap[channel])
    fig.colorbar(datadisp)

    plt.show()

    # Display RGB
    print(channumber,'channels combined at the given timeframe t =',time,'and z-position =',zpos)

    shp = (myimg.shape[3],myimg.shape[4],channumber) # tuple
    rgb = np.zeros(shp,dtype=myimg.dtype)

    for k in np.arange(channumber):
        rgb[:,:,k] = myimg[time,k,zpos,:,:]
    # set the range of the rgb data to be between 0 and 1 to avoid clipping
    maxValue = np.amax(rgb)
    minValue = np.amin(rgb)
    lb=0; # lower bound equal to 0
    if lb==0:
        s=1
    else:
        s=-np.sign(lb) # this should not be an option since we are dealing with intensity so the lower bound in the original data should always be positive but for future use, keep it as it is!
    ub=1; # upper bound equal to 1
    rgbnew=(rgb-minValue)/(maxValue - minValue)*(s*np.abs(lb)+np.abs(ub))-s*np.abs(lb)


    fig, ax2 = plt.subplots()
    datadisp2 = ax2.imshow(rgbnew)
    fig.colorbar(datadisp2)
    plt.show()


    return()
print('Slide the timeframe and z-position and select the desired channel to display: ')
interact(series, time=(0,imgsize[0]-1,1),zpos=(0,imgsize[2]-1,1), channel=channelname);

Slide the timeframe and z-position and select the desired channel to display: 


interactive(children=(IntSlider(value=0, description='time', max=0), IntSlider(value=50, description='zpos'), …